In [1]:
import numpy as np
import scipy
from numpy import mean, absolute
import pandas as pd
from sklearn import *

from scipy import spatial
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import distance
import math
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from itertools import chain
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [3]:
from os import listdir
from pathlib import Path
from os.path import isfile, join
from natsort import natsorted ## !pip install natsort

### Loading the data in dataframe

In [4]:
data=pd.read_csv ('AWID3_clean_28.csv', low_memory=False)
data.head()

,Label,frame.len,frame.number,frame.time_delta,frame.time_delta_displayed,frame.time_epoch,frame.time_relative,radiotap.datarate,radiotap.dbm_antsignal,radiotap.length,...,wlan.fc.pwrmgt,wlan.fc.retry,wlan.fc.subtype,wlan.fc.type,wlan.ra,wlan.ta,wlan_radio.data_rate,wlan_radio.duration,wlan_radio.phy,wlan_radio.signal_dbm
0,0,342.0,1.0,0.000000,0.000000,1.608306e+09,0.000000,6.0,-34-37-34,56.0,...,0.0,0.0,8.0,0.0,1,1,6.0,408.0,5.0,-34.0
1,0,342.0,2.0,0.102410,0.102410,1.608306e+09,0.102410,6.0,-34-38-34,56.0,...,0.0,0.0,8.0,0.0,1,1,6.0,408.0,5.0,-34.0
2,0,136.0,3.0,0.044786,0.044786,1.608306e+09,0.147196,6.0,-34-39-34,56.0,...,0.0,0.0,0.0,2.0,1,1,6.0,132.0,5.0,-34.0
3,0,154.0,4.0,0.007499,0.007499,1.608306e+09,0.154695,6.0,-34-39-34,56.0,...,0.0,0.0,0.0,2.0,1,1,6.0,156.0,5.0,-34.0
4,0,342.0,5.0,0.050126,0.050126,1.608306e+09,0.204821,6.0,-35-40-35,56.0,...,0.0,0.0,8.0,0.0,1,1,6.0,408.0,5.0,-35.0


In [5]:
print(data.shape)

(36962587, 28)


### Split the data in training, val and testing

In [6]:
X_df=data.iloc[:,1:]
Y_df=data.iloc[:,0]
print(X_df.shape)
print(Y_df.shape)

(36962587, 27)
(36962587,)


In [7]:
Y_df.iloc[:].value_counts(dropna=False)

0     30387097
11     5499851
13      405121
6       191803
9       131611
12      104827
2        75131
8        56891
5        49990
1        38942
7        11882
3         5502
10        2629
4         1310
Name: Label, dtype: int64

In [8]:
X_df["radiotap.dbm_antsignal"] = X_df["radiotap.dbm_antsignal"].astype('category')
X_df["radiotap.dbm_antsignal"] = X_df["radiotap.dbm_antsignal"].cat.codes
X_df.iloc[:,7].unique()

array([524, 525, 526, ..., 837, 598,  65], dtype=int16)

In [9]:
from sklearn.model_selection import train_test_split

X_train_, X_temp, y_train_, y_temp = train_test_split(X_df,
                                                    Y_df,
                                                    test_size=0.05,
                                                    random_state=5,
                                                    stratify=Y_df
                                                    )

X_train, X_test, y_train, y_test = train_test_split(X_temp,
                                                    y_temp,
                                                    test_size=0.1,
                                                    random_state=5,
                                                    stratify=y_temp
                                                    )


In [10]:
print(X_train.shape , X_test.shape)
print(y_train.shape , y_test.shape)

(1663317, 27) (184813, 27)
(1663317,) (184813,)


In [11]:
y_train.iloc[:].value_counts(dropna=False)

0     1367420
11     247494
13      18230
6        8631
9        5923
12       4717
2        3381
8        2560
5        2250
1        1752
7         535
3         247
10        118
4          59
Name: Label, dtype: int64

In [12]:
y_test.iloc[:].value_counts(dropna=False)

0     151935
11     27499
13      2026
6        959
9        658
12       524
2        376
8        285
5        250
1        195
7         59
3         28
10        13
4          6
Name: Label, dtype: int64

In [13]:
for i in X_train:
    print(i+" "+str(len(X_train[i].unique())))
    #print(len(df2[i].unique()))

frame.len 1756
frame.number 1382465
frame.time_delta 10391
frame.time_delta_displayed 10391
frame.time_epoch 195786
frame.time_relative 1660569
radiotap.datarate 30
radiotap.dbm_antsignal 1473
radiotap.length 3
radiotap.present.tsft 2
radiotap.timestamp.ts 1424518
wlan.duration 2859
wlan.fc.ds 4
wlan.fc.frag 2
wlan.fc.moredata 2
wlan.fc.order 2
wlan.fc.protected 2
wlan.fc.pwrmgt 2
wlan.fc.retry 2
wlan.fc.subtype 14
wlan.fc.type 3
wlan.ra 1
wlan.ta 2
wlan_radio.data_rate 36
wlan_radio.duration 718
wlan_radio.phy 5
wlan_radio.signal_dbm 66


In [22]:
xtrainFileName="xtrain_AWID_28_reduced.npy"
xtestFileName="xtest_AWID_28_reduced.npy"
ytrainFileName="ytrain_AWID_28_reduced.npy"
ytestFileName="ytest_AWID_28_reduced.npy"
np.save(xtrainFileName, X_train)
np.save(xtestFileName, X_test)
np.save(ytrainFileName, y_train)
np.save(ytestFileName, y_test)

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [15]:
scaler=StandardScaler().fit(X_train)
X_train_transformed=scaler.transform(X_train)
X_test_transformed=scaler.transform(X_test)

In [16]:
X_train_transformed

array([[-0.57244847, -0.83217363, -0.07427899, ..., -0.47002302,
        -0.97198544, -0.76613581],
       [-0.60199699, -0.01866551, -0.09213789, ..., -0.47002302,
        -0.97198544, -1.4083291 ],
       [-0.60199699,  0.04429943, -0.08009351, ..., -0.28251807,
        -0.97198544,  0.51825079],
       ...,
       [ 1.82754774, -0.64503608, -0.07012576, ...,  0.46750174,
         1.05597342,  1.01773447],
       [ 1.87679527, -0.79560333,  0.04035024, ...,  0.67844481,
         0.37998714,  1.08908928],
       [-0.45753757, -0.63329279, -0.04146845, ..., -0.28251807,
         1.05597342, -0.76613581]])

### First ML Model - Bagging

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import itertools
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
import time

In [18]:
seed_lst = [j*7 for j in range(3)]
mean_scores = []
start = time.time()
for seed in seed_lst:
    model = BaggingClassifier(random_state = seed)
    scores = cross_val_score(model, X_train_transformed, y_train, 
                             cv=5, scoring='accuracy', verbose=2)
    score = scores.mean()*100
    mean_scores.append(score)   

mean_res3 = np.array(mean_scores)
print('Accuracy: %.4f(+/-%.4f)'%(mean_res3.mean(), mean_res3.std()))
end = time.time()
print('Time elapsed: %.4f'%(end-start))  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time= 2.2min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.2min remaining:    0.0s


[CV] END .................................................... total time= 2.2min
[CV] END .................................................... total time= 2.2min
[CV] END .................................................... total time= 2.2min
[CV] END .................................................... total time= 2.2min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 10.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 10.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time= 2.2min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.2min remaining:    0.0s


[CV] END .................................................... total time= 2.2min
[CV] END .................................................... total time= 2.1min
[CV] END .................................................... total time= 2.2min
[CV] END .................................................... total time= 2.2min
Accuracy: 99.8017(+/-0.0016)
Time elapsed: 1955.3290


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 10.8min finished


In [19]:
from sklearn.ensemble import ExtraTreesClassifier

In [20]:
seed_lst = [j*7 for j in range(3)]
mean_scores = []
start = time.time()
for seed in seed_lst:
    model = ExtraTreesClassifier(random_state = seed)
    scores = cross_val_score(model, X_train_transformed, y_train, cv=5, scoring='accuracy',verbose=2)
    score = scores.mean()*100
    mean_scores.append(score)   

mean_res1 = np.array(mean_scores)
print('Accuracy: %.4f(+/-%.4f)'%(mean_res1.mean(), mean_res1.std()))
end = time.time()
print('Time elapsed: %.4f'%(end-start)) 

Accuracy: 99.7929(+/-0.0007)
Time elapsed: 3360.7512


In [24]:
seed_lst = [j*7 for j in range(3)]
mean_scores = []
start = time.time()
for seed in seed_lst:
    model = RandomForestClassifier(n_estimators =10, min_samples_leaf = 1, min_samples_split = 2, random_state = seed)
    scores = cross_val_score(model, X_train_transformed, y_train, cv=5, scoring='accuracy',verbose=2)###cv=10 just 2 for PSO
    score = scores.mean()*100
    mean_scores.append(score)   

mean_res1 = np.array(mean_scores)
print('Accuracy: %.4f(+/-%.4f)'%(mean_res1.mean(), mean_res1.std()))
end = time.time()
print('Time elapsed: %.4f'%(end-start))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=  34.9s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   35.2s remaining:    0.0s


[CV] END .................................................... total time=  34.6s
[CV] END .................................................... total time=  33.1s
[CV] END .................................................... total time=  34.9s
[CV] END .................................................... total time=  33.6s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.0min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=  35.1s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   35.3s remaining:    0.0s


[CV] END .................................................... total time=  35.5s
[CV] END .................................................... total time=  34.4s
[CV] END .................................................... total time=  36.9s
[CV] END .................................................... total time=  33.7s
Accuracy: 99.7888(+/-0.0018)
Time elapsed: 529.3805


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.9min finished


In [26]:
import xgboost as xgb

In [27]:
seed_lst = [j*7 for j in range(3)]
mean_scores = []
start = time.time()
for seed in seed_lst:
    model = xgb.XGBClassifier(max_depth=10, n_estimators=3, learning_rate=0.01, seed=seed)
    scores = cross_val_score(model, X_train_transformed, y_train, cv=5, scoring='accuracy',verbose=2)
    score = scores.mean()*100
    mean_scores.append(score)   

mean_res1 = np.array(mean_scores)
print('Accuracy: %.4f(+/-%.4f)'%(mean_res1.mean(), mean_res1.std()))
end = time.time()
print('Time elapsed: %.4f'%(end-start)) 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\sanjay\anaconda3\envs\Projects_Segmentation\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:05:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END .................................................... total time=  18.4s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.7s remaining:    0.0s
C:\Users\sanjay\anaconda3\envs\Projects_Segmentation\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:05:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END .................................................... total time=  18.7s


C:\Users\sanjay\anaconda3\envs\Projects_Segmentation\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\sanjay\anaconda3\envs\Projects_Segmentation\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:09:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END .................................................... total time=  18.6s


C:\Users\sanjay\anaconda3\envs\Projects_Segmentation\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:09:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END .................................................... total time=  19.2s
Accuracy: 99.7037(+/-0.0000)
Time elapsed: 285.3340


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.6min finished


In [29]:
from sklearn.naive_bayes import GaussianNB

In [32]:
seed_lst = [j*7 for j in range(3)]
mean_scores = []
start = time.time()
for seed in seed_lst:
    model = GaussianNB()
    scores = cross_val_score(model, X_train_transformed, y_train, cv=5, scoring='accuracy',verbose=2)
    score = scores.mean()*100
    mean_scores.append(score)   

mean_res1 = np.array(mean_scores)
print('Accuracy: %.4f(+/-%.4f)'%(mean_res1.mean(), mean_res1.std()))
end = time.time()
print('Time elapsed: %.4f'%(end-start))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   1.9s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


[CV] END .................................................... total time=   2.0s
[CV] END .................................................... total time=   1.9s
[CV] END .................................................... total time=   1.9s
[CV] END .................................................... total time=   1.9s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   10.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   10.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   1.9s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


[CV] END .................................................... total time=   1.9s
[CV] END .................................................... total time=   1.9s
[CV] END .................................................... total time=   2.0s
[CV] END .................................................... total time=   1.9s
Accuracy: 87.0797(+/-0.0000)
Time elapsed: 31.4868


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   10.4s finished


In [33]:
from sklearn.tree import DecisionTreeClassifier

In [34]:
seed_lst = [j*7 for j in range(3)]
mean_scores = []
start = time.time()
for seed in seed_lst:
    model = DecisionTreeClassifier(random_state=seed, max_depth=2)
    scores = cross_val_score(model, X_train_transformed, y_train, cv=5, scoring='accuracy',verbose=2)
    score = scores.mean()*100
    mean_scores.append(score)   

mean_res1 = np.array(mean_scores)
print('Accuracy: %.4f(+/-%.4f)'%(mean_res1.mean(), mean_res1.std()))
end = time.time()
print('Time elapsed: %.4f'%(end-start))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   5.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.4s remaining:    0.0s


[CV] END .................................................... total time=   5.2s
[CV] END .................................................... total time=   5.2s
[CV] END .................................................... total time=   5.2s
[CV] END .................................................... total time=   5.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   26.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   25.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   5.1s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.3s remaining:    0.0s


[CV] END .................................................... total time=   5.2s
[CV] END .................................................... total time=   5.2s
[CV] END .................................................... total time=   5.2s
[CV] END .................................................... total time=   5.2s
Accuracy: 94.3283(+/-0.0000)
Time elapsed: 79.0468


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   26.7s finished


In [36]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
seed_lst = [j for j in range(10)]
mean_scores = []
start = time.time()
for seed in seed_lst:
    model = KNeighborsClassifier(n_neighbors=seed)
    scores = cross_val_score(model, X_train_transformed, y_train, cv=5, scoring='accuracy',verbose=2)
    score = scores.mean()*100
    mean_scores.append(score)   

mean_res1 = np.array(mean_scores)
print('Accuracy: %.4f(+/-%.4f)'%(mean_res1.mean(), mean_res1.std()))
end = time.time()
print('Time elapsed: %.4f'%(end-start))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.6s finished
C:\Users\sanjay\anaconda3\envs\Projects_Segmentation\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\sanjay\anaconda3\envs\Projects_Segmentation\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\sanjay\anaconda3\envs\Projects_Segmentation\lib\site-packages\sklearn\neighbors\_classification.py", line 198, in fit
    return self._fit(X, y)
  File "C:\Users\sanjay\a

In [ ]:
seed_lst = [j*7 for j in range(3)]
mean_scores = []
start = time.time()
for seed in seed_lst:
    model = AdaBoostClassifier(random_state=seed, n_estimators=10)
    scores = cross_val_score(model, X_train_transformed, y_train, cv=5, scoring='accuracy',verbose=2)
    score = scores.mean()*100
    mean_scores.append(score)   

mean_res1 = np.array(mean_scores)
print('Accuracy: %.4f(+/-%.4f)'%(mean_res1.mean(), mean_res1.std()))
end = time.time()
print('Time elapsed: %.4f'%(end-start))